# Test script for your environment

This jupyter notebook helps to test the environment. It is a basic "Hello World" equivalent for the HuggingFace. 

It imports the right libraries, downloads the right model and runs a simple test.

When you run this script from Visual Studio code, make sure that you use the right interpreter -- the one that is in your virtual environment. You can add it via the command palette `(Ctrl+Shift+P)` and then `Python: Select Interpreter` -- select the one that is in your virtual environment (or navigate to the right path to add it -- do NOT create a new virutal environment. )

In [1]:
# importing os to ensure that we do not get too many unrelated warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['TOKENIZERS_PARALLELISM'] = 'true'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'

# stop warnings
import warnings
warnings.filterwarnings("ignore")

In [14]:
# this code checks if the transformers library is installed
# the output should be the version of the transformers library
import transformers

print(transformers.__version__)

4.38.2


In [15]:
# the same for the torch library
import torch

print(torch.__version__)

2.2.1+cu121


In [16]:
# and for the tensorflow library
import tensorflow as tf

print(tf.__version__)

2.15.0


## Smaller model

The model below is a very small one - ca. 85 million parameters. This means that it should run on all environments. So, this model should not fail on your computer. If it does, then you need to debug your environment and you need to make it work. 

In [3]:
# this tests a simple RoBERTa model trained by our research group on Singletons
from transformers import pipeline

unmasker = pipeline('fill-mask', model='mstaron/SingletonBERT')

unmasker("Hello I'm a <mask> model.")

[{'score': 0.36955469846725464,
  'token': 13963,
  'token_str': ' sliding',
  'sequence': "Hello I'm a sliding model."},
 {'score': 0.1200207844376564,
  'token': 11430,
  'token_str': ' working',
  'sequence': "Hello I'm a working model."},
 {'score': 0.07376236468553543,
  'token': 1464,
  'token_str': ' new',
  'sequence': "Hello I'm a new model."},
 {'score': 0.06340111047029495,
  'token': 1258,
  'token_str': ' into',
  'sequence': "Hello I'm a into model."},
 {'score': 0.03293626010417938,
  'token': 3596,
  'token_str': ' projective',
  'sequence': "Hello I'm a projective model."}]

## Larger model

Below we test a bit of a larger model -- it can take a bit of time to download and run.

If you do not have a GPU, you can remove the .to('cuda') line from the code below. 

If it does not work at all, do not worry, it may just mean that your computer is too small for the larger models. That's also fine and you can still work with the smaller models.

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct", trust_remote_code=True, torch_dtype=torch.bfloat16).to("cuda")
messages=[
    { 'role': 'user', 'content': "write a quick sort algorithm in python."}
]
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
# tokenizer.eos_token_id is the id of <|EOT|> token
outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/mnt/d/venvs/hackathon_genAI/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Sure, here is a simple implementation of the Quick Sort algorithm in Python:

```python
def quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quicksort(left) + middle + quicksort(right)

# Test the function
print(quicksort([3,6,8,10,1,2,1]))
# Output: [1, 1, 2, 3, 6, 8, 10]
```

This algorithm works by selecting a 'pivot' element from the array and partitioning the other elements into two sub-arrays, according to whether they are less than or greater than the pivot. The sub-arrays are then recursively sorted.

